### Import Libraries

In [1]:
import pickle
import os
import pandas as pd
import enum
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from spear.labeling import PreLabels
import numpy as np
from spear.cage import Cage
from utils import custom_dataset, train_all_LF, class_count
from generate_LF import get_variables2
from spear.labeling import labeling_function, ABSTAIN, preprocessor, continuous_scorer
from cnn import create_cnn
from keras.utils import np_utils
from spear.labeling import labeling_function, ABSTAIN, preprocessor, continuous_scorer
import re
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn import metrics
import math

c:\Users\adity\Documents\GitHub\MICCAI\med\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/25
1/1 [==============================] - 1s 1s/step - loss: 1.0823 - accuracy: 0.3500 - val_loss: 1.1402 - val_accuracy: 0.0000e+00
Epoch 2/25
1/1 [==============================] - 0s 91ms/step - loss: 1.0457 - accuracy: 0.4750 - val_loss: 1.2024 - val_accuracy: 0.0000e+00
Epoch 3/25
1/1 [==============================] - 0s 72ms/step - loss: 1.0517 - accuracy: 0.4000 - val_loss: 1.2424 - val_accuracy: 0.0000e+00
Epoch 4/25
1/1 [==============================] - 0s 71ms/step - loss: 1.0940 - accuracy: 0.3250 - val_loss: 1.2289 - val_accuracy: 0.0000e+00
Epoch 5/25
1/1 [==============================] - 0s 68ms/step - loss: 1.0249 - accuracy: 0.3750 - val_loss: 1.2056 - val_accuracy: 0.0000e+00
Epoch 6/25
1/1 [==============================] - 0s 58ms/step - loss: 1.0333 - accuracy: 0.4250 - val_loss: 1.1640 - val_accuracy: 0.0000e+00
Epoch 7/25
1/1 [==============================] - 0s 60ms/step - loss: 0.9895 - accuracy: 0.5750 - val_loss: 1.1074 - val_accuracy: 0.0000e+00
E

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Declaring ClassLabels

In [3]:
ABSTAIN = None

class ClassLabels(enum.Enum):
    BASOPHIL = 0
    EOSINOPHIL = 1
    ERYTHROBLAST = 2
    IMG = 3
    LYMPHOCYTE = 4
    MONOCYTE = 5
    NEUTROPHIL = 6
    PLATELET = 7


### Defining Labelling Functions

In [4]:
path = "C:\\Users\\adity\\Documents\\GitHub\\MICCAI\\data2\\models\\"

In [5]:
# SVM
@continuous_scorer()
def svm_0(x,**kwargs):
    x = np.array(x).flatten() # x is 28x28x3 input img 
    svm = pickle.load(open(path+'0_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    # print(confidence_scores)
    return float(confidence_scores[0][1]) #only called when model doesn't ABSTAIN, output confidence on class label

@labeling_function(cont_scorer=svm_0, label=ClassLabels.BASOPHIL)
def LF_svm_0(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'0_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.BASOPHIL # Return label only if confidence > 0.8
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_0(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'0_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_0, label=ClassLabels.BASOPHIL)
def LF_rf_0(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'0_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.BASOPHIL
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_0(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'0_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_0, label=ClassLabels.BASOPHIL)
def LF_knn_0(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'0_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.BASOPHIL
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_0(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'0_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_0, label=ClassLabels.BASOPHIL)
def LF_dt_0(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'0_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.BASOPHIL
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_0(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'0_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_0, label=ClassLabels.BASOPHIL)
def LF_lr_0(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'0_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.BASOPHIL
    else: 
        return ABSTAIN


In [6]:
# SVM
@continuous_scorer()
def svm_1(x,**kwargs):
    x = np.array(x).flatten() # x is 28x28x3 input img 
    svm = pickle.load(open(path+'1_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1]) #only called when model doesn't ABSTAIN, output confidence on class label

@labeling_function(cont_scorer=svm_1, label=ClassLabels.EOSINOPHIL)
def LF_svm_1(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'1_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.EOSINOPHIL
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_1(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'1_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_1, label=ClassLabels.EOSINOPHIL)
def LF_rf_1(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'1_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.EOSINOPHIL
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_1(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'1_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_1, label=ClassLabels.EOSINOPHIL)
def LF_knn_1(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'1_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.EOSINOPHIL
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_1(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'1_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_1, label=ClassLabels.EOSINOPHIL)
def LF_dt_1(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'1_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.EOSINOPHIL
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_1(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'1_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_1, label=ClassLabels.EOSINOPHIL)
def LF_lr_1(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'1_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.EOSINOPHIL
    else: 
        return ABSTAIN


In [7]:
# SVM
@continuous_scorer()
def svm_2(x,**kwargs):
    x = np.array(x).flatten()   
    svm = pickle.load(open(path+'2_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1])  

@labeling_function(cont_scorer=svm_2, label=ClassLabels.ERYTHROBLAST)
def LF_svm_2(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'2_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.ERYTHROBLAST
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_2(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'2_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_2, label=ClassLabels.ERYTHROBLAST)
def LF_rf_2(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'2_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.ERYTHROBLAST
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_2(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'2_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_2, label=ClassLabels.ERYTHROBLAST)
def LF_knn_2(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'2_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.ERYTHROBLAST
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_2(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'2_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_2, label=ClassLabels.ERYTHROBLAST)
def LF_dt_2(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'2_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.ERYTHROBLAST
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_2(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'2_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_2, label=ClassLabels.ERYTHROBLAST)
def LF_lr_2(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'2_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.ERYTHROBLAST
    else: 
        return ABSTAIN


In [8]:
# SVM
@continuous_scorer()
def svm_3(x,**kwargs):
    x = np.array(x).flatten()   
    svm = pickle.load(open(path+'3_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1])  

@labeling_function(cont_scorer=svm_3, label=ClassLabels.IMG)
def LF_svm_3(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'3_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.IMG
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_3(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'3_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_3, label=ClassLabels.IMG)
def LF_rf_3(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'3_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.IMG
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_3(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'3_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_3, label=ClassLabels.IMG)
def LF_knn_3(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'3_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.IMG
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_3(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'3_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_3, label=ClassLabels.IMG)
def LF_dt_3(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'3_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.IMG
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_3(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'3_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_3, label=ClassLabels.IMG)
def LF_lr_3(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'3_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.IMG
    else: 
        return ABSTAIN


In [9]:
# SVM
@continuous_scorer()
def svm_4(x,**kwargs):
    x = np.array(x).flatten()   
    svm = pickle.load(open(path+'4_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1])  

@labeling_function(cont_scorer=svm_4, label=ClassLabels.LYMPHOCYTE)
def LF_svm_4(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'4_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.LYMPHOCYTE
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_4(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'4_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_4, label=ClassLabels.LYMPHOCYTE)
def LF_rf_4(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'4_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.LYMPHOCYTE
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_4(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'4_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_4, label=ClassLabels.LYMPHOCYTE)
def LF_knn_4(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'4_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.LYMPHOCYTE
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_4(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'4_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_4, label=ClassLabels.LYMPHOCYTE)
def LF_dt_4(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'4_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.LYMPHOCYTE
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_4(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'4_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_4, label=ClassLabels.LYMPHOCYTE)
def LF_lr_4(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'4_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.LYMPHOCYTE
    else: 
        return ABSTAIN


In [10]:
# SVM
@continuous_scorer()
def svm_5(x,**kwargs):
    x = np.array(x).flatten()   
    svm = pickle.load(open(path+'5_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1])  

@labeling_function(cont_scorer=svm_5, label=ClassLabels.MONOCYTE)
def LF_svm_5(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'5_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.MONOCYTE
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_5(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'5_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_5, label=ClassLabels.MONOCYTE)
def LF_rf_5(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'5_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.MONOCYTE
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_5(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'5_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_5, label=ClassLabels.MONOCYTE)
def LF_knn_5(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'5_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.MONOCYTE
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_5(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'5_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_5, label=ClassLabels.MONOCYTE)
def LF_dt_5(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'5_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.MONOCYTE
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_5(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'5_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_5, label=ClassLabels.MONOCYTE)
def LF_lr_5(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'5_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.MONOCYTE
    else: 
        return ABSTAIN


In [11]:
# SVM
@continuous_scorer()
def svm_6(x,**kwargs):
    x = np.array(x).flatten()   
    svm = pickle.load(open(path+'6_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1])  

@labeling_function(cont_scorer=svm_6, label=ClassLabels.NEUTROPHIL)
def LF_svm_6(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'6_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.NEUTROPHIL
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_6(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'6_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_6, label=ClassLabels.NEUTROPHIL)
def LF_rf_6(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'6_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.NEUTROPHIL
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_6(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'6_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_6, label=ClassLabels.NEUTROPHIL)
def LF_knn_6(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'6_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.NEUTROPHIL
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_6(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'6_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_6, label=ClassLabels.NEUTROPHIL)
def LF_dt_6(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'6_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.NEUTROPHIL
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_6(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'6_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_6, label=ClassLabels.NEUTROPHIL)
def LF_lr_6(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'6_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.NEUTROPHIL
    else: 
        return ABSTAIN


In [12]:
# SVM
@continuous_scorer()
def svm_7(x,**kwargs):
    x = np.array(x).flatten()   
    svm = pickle.load(open(path+'7_svm.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    return float(confidence_scores[0][1])  

@labeling_function(cont_scorer=svm_7, label=ClassLabels.PLATELET)
def LF_svm_7(x, **kwargs):
    x = np.array(x).flatten()  
    svm = pickle.load(open(path+'7_svm.pkl','rb'))
    if svm.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.PLATELET
    else: 
        return ABSTAIN

# Random Forest
@continuous_scorer()
def rf_7(x,**kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'7_rf.pkl','rb'))
    confidence_scores = rf.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=rf_7, label=ClassLabels.PLATELET)
def LF_rf_7(x, **kwargs):
    x = np.array(x).flatten()  
    rf = pickle.load(open(path+'7_rf.pkl','rb'))
    if rf.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.PLATELET
    else: 
        return ABSTAIN

# KNN
@continuous_scorer()
def knn_7(x,**kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'7_knn.pkl','rb'))
    confidence_scores = knn.predict_proba([x])
    return float(confidence_scores[0][1]) 


@labeling_function(cont_scorer=knn_7, label=ClassLabels.PLATELET)
def LF_knn_7(x, **kwargs):
    x = np.array(x).flatten()  
    knn = pickle.load(open(path+'7_knn.pkl','rb'))
    if knn.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.PLATELET
    else: 
        return ABSTAIN

# Decision Tree 
@continuous_scorer()
def dt_7(x,**kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'7_dtc.pkl','rb'))
    confidence_scores = dt.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=dt_7, label=ClassLabels.PLATELET)
def LF_dt_7(x, **kwargs):
    x = np.array(x).flatten()  
    dt = pickle.load(open(path+'7_dtc.pkl','rb'))
    if dt.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.PLATELET
    else: 
        return ABSTAIN

# Logistic Regression
@continuous_scorer()
def lr_7(x,**kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'7_lr.pkl','rb'))
    confidence_scores = lr.predict_proba([x])
    return float(confidence_scores[0][1]) 

@labeling_function(cont_scorer=lr_7, label=ClassLabels.PLATELET)
def LF_lr_7(x, **kwargs):
    x = np.array(x).flatten()  
    lr = pickle.load(open(path+'7_lr.pkl','rb'))
    if lr.predict_proba([x])[0][1]>0.8: 
        return ClassLabels.PLATELET
    else: 
        return ABSTAIN


## Aggregating LFs & Labelling dataset 

### LFSet
Placeholder for declared LFs

In [13]:
from spear.labeling import LFSet

LFS = [
    LF_svm_0,
    LF_rf_0,
    LF_knn_0,
    LF_dt_0,
    LF_lr_0,
    LF_svm_1,
    LF_rf_1,
    LF_knn_1,
    LF_dt_1,
    LF_lr_1,
    LF_svm_2,
    LF_rf_2,
    LF_knn_2,
    LF_dt_2,
    LF_lr_2,
    LF_svm_3,
    LF_rf_3,
    LF_knn_3,
    LF_dt_3,
    LF_lr_3,
    LF_svm_4,
    LF_rf_4,
    LF_knn_4,
    LF_dt_4,
    LF_lr_4,
    LF_svm_5,
    LF_rf_5,
    LF_knn_5,
    LF_dt_5,
    LF_lr_5,
    LF_svm_6,
    LF_rf_6,
    LF_knn_6,
    LF_dt_6,
    LF_lr_6,
    LF_svm_7,
    LF_rf_7,
    LF_knn_7,
    LF_dt_7,
    LF_lr_7,
]


rules = LFSet("BM_LF")
rules.add_lf_list(LFS)

### Load the dataset

In [14]:
from utils import custom_dataset, train_all_LF
from generate_LF import get_variables2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

classes,label_frac,data_path,save_path = get_variables2()
dataset,x,y = custom_dataset(classes=classes,path=data_path ,fraction=label_frac)
yOracle = copy.deepcopy(y)

In [15]:
def map (curr, cls):
    if curr==cls:
        return 1
    else:
        return 0

### CNN

In [16]:
def cnnScore(x, y,i):
    x_train = np.array(x).reshape(-1, 28, 28, 3)
    x_train = x_train.astype("float32") / 255
    y_train = [int(i) for i in y]
    y_train = np_utils.to_categorical(y_train, len(classes))

    # Load Validation Data
    # 5% of Val Set to Test CNN after every iteration
    x_val, dummy1, y_val, dummy2 = train_test_split(dataset["val_images"], dataset["val_labels"], train_size=0.2)
    x_val = np.array(x_val).reshape(-1, 28, 28, 3)
    x_val = x_val.astype("float32") / 255
    y_val = [int(i) for i in y_val] 
    
    xv = []
    yv = []
    class_limit = int(0.05*len(dataset["val_labels"])/len(classes))
    for j in range(len(classes)):
        for i in range(len(y_val)):
            if  yv.count(j) < class_limit and y_val[i] == j:
                xv.append(x_val[i])
                yv.append(y_val[i])

    # print('*'*80)
    # print([yv.count(i) for i in range(len(classes))])

    xv = np.array(xv)
    yv = np.array(yv)
    yv = np_utils.to_categorical(yv, num_classes=len(classes))

    batch_size = 128
    epochs = 25
    model = create_cnn(num_classes = len(classes))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose = 0)
    return model.evaluate(xv, yv, verbose = 0)

## Cage

In [23]:
def cage_loop(LFS, max_iters=10, threshold=10**-5, img_per_class = 150):
    # Paths
    log_path_cage = './cage_loop2/log.txt' 
    params_path = None
    path_json = "./cage_loop2/labels.json"
    U_path_pkl = "./cage_loop2/unlabelled.pkl"
    L_path_pkl = "./cage_loop2/labelled.pkl"

    # Loading Data
    classes,label_frac,data_path,save_path = get_variables2()
    dataset,x,y = custom_dataset(classes=classes, path=data_path, fraction=label_frac)
    xu = np.array(dataset['rem_images'])
    yu = np.array(dataset['rem_labels'])
    yOracle = copy.deepcopy(y)
    
    print("Classes used in expt:",classes)

    # Creating rules
    n_lfs = len(LFS)
    rules = LFSet("BM_LF")
    rules.add_lf_list(LFS)
    
    confidence_list = []
    val_scores = []
    oracle_val_scores = []
    classwise_accuracies = []
    pl_accuracies = []
    for i in range(max_iters):
        # Train Models in LFs
        train_all_LF(x,y,len(classes),save_path,label_frac)

        # Unlabelled
        u_noisy_labels = PreLabels(name="bmnist_rem_ul",
                                    data=xu,
                                    rules=rules,
                                    labels_enum=ClassLabels,
                                    num_classes=len(classes))
        # Lu,Su = u_noisy_labels.get_labels()
        # if i!=0:
        u_noisy_labels.generate_pickle(U_path_pkl)
        u_noisy_labels.generate_json(path_json)

        # Labelled
        # l_noisy_labels = PreLabels(name="bmnist_l",
        #                             data=x,
        #                             gold_labels=y,
        #                             rules=rules,
        #                             labels_enum=ClassLabels,
        #                             num_classes=len(classes))
        # # Ll,Sl = l_noisy_labels.get_labels()
        # l_noisy_labels.generate_pickle(L_path_pkl)
        # l_noisy_labels.generate_json(path_json)


        # Cage
        cage = Cage(path_json = path_json, n_lfs = n_lfs)
        if params_path is not None: 
            cage.load_params(load_path = params_path)
        else:
            params_path = './cage_loop2/params.pkl' 
        
        probs = cage.fit_and_predict_proba(path_pkl = U_path_pkl, path_log = log_path_cage, qt = 0.9, qc = 0.85, metric_avg = ['macro'], n_epochs = 100, lr = 0.01)
        labels = np.argmax(probs, 1)

        print("="*135)
        print("Iteration",i)
        values, frequency = np.unique(yu, return_counts=True)
        for values, frequency in zip(values, frequency):
            print(f"Labels of Lake Class {values}: {frequency}")

        values, frequency = np.unique(y, return_counts=True)
        for values, frequency in zip(values, frequency):
            print(f"Labels of Labelled Set {values}: {frequency}")
        
        
        
        print("Shape of Labeled Data:",x.shape)
        print("Shape of Unlabeled Data:",xu.shape)
        print("Accuracy on unlabelled images:",accuracy_score(labels,yu)*100)
        
        
        cage.save_params(save_path = params_path)

        confidence = np.array([np.max(i) for i in probs])
        confidence_list.append(confidence)
        print(i,probs.shape)

        # Getting indices of probabilities in decreasing order
        idx = np.argsort(confidence)
        idx = idx[::-1] 

        # Number of images per class (5%)
        # img_per_class = int(0.05*len(confidence)/len(classes))

        # Number of images per class (50)
        
        
        print("Num img per class =",img_per_class)

        pop_list = [] #list of indices of images to be added
        label_count = []
        threshcount = 0
        unl = []
        for j in idx:
            if confidence[j]>threshold:
                threshcount+=1
                unl.append(labels[j])

            if confidence[j]>threshold and label_count.count(labels[j])<img_per_class:
                pop_list.append(j)
                label_count.append(labels[j])
        
        print("Number of images above threshold:",threshcount)
        print("Classes of images above threshold:",set(unl))
        print("Number of images getting transferred:", len(pop_list))
        print('Accuracy of Pseudo-labelled img added to dataset:', accuracy_score(labels[pop_list],yu[pop_list])*100)
        pl_accuracies.append(accuracy_score(labels[pop_list],yu[pop_list])*100)

        # Confusion Matrix
        cmidx = [[(map(yu[j],i), map(labels[j],i)) for j in range(len(labels))] for i in range(len(classes))]
        cacc = []
        fig, ax = plt.subplots(math.ceil(len(classes)/2),2, figsize=(15, 20))
        pltnum = 0
        for cmid in cmidx:
            cacc.append(accuracy_score([z[0] for z in cmid], [z[1] for z in cmid])*100)
            ax[pltnum//2,pltnum%2].set_title(f"Class {pltnum}")
            confusion_matrix = metrics.confusion_matrix([z[0] for z in cmid], [z[1] for z in cmid])
            cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
            cm_display.plot(ax=ax[pltnum//2,pltnum%2])
            pltnum += 1
        classwise_accuracies.append(cacc)
        fig.suptitle(f"Iteration {i}")
        plt.show()
        plt.savefig(f"all_cm_iteration_{i}.jpg")

        if len(pop_list)<50:
            break
        
        x = np.append(x,xu[pop_list], axis=0)
        y = np.append(y,labels[pop_list], axis=0)
        yOracle = np.append(yOracle,yu[pop_list], axis=0)
        xu = np.delete(xu,pop_list, axis=0)
        yu = np.delete(yu,pop_list, axis=0)

        # Deleting variables
        del u_noisy_labels
        del cage

        classes,label_frac,data_path,save_path = get_variables2()

        val_scores.append(cnnScore(x, y,i)[1]*100)
        oracle_val_scores.append(cnnScore(x,yOracle,i)[1]*100)
        print(f"CNN Val accuracy trained on Lake Set for iteration {i}: ", val_scores[i])
        print(f"CNN Val accuracy trained on Oracle for iteration {i}: ", oracle_val_scores[i])

        # for j in range(len(classes)):
        #     if label_count.count(j)<75:
        #         print("*"*60+"Predicted Break"+"*"*60)
        #         break

        
        print("="*135)
        # if i>0 and val_scores[i]<val_scores[i-1]:
        #     break


    return x,y,xu,yu,confidence_list, val_scores, oracle_val_scores, classwise_accuracies, pl_accuracies


In [24]:
#  img_per_class: num images added per loop per class
x,y,xu,yu,confidence_list, val_scores, oracle_val_scores, classwise_accuracies, pl_accuracies = cage_loop(LFS, max_iters=1, threshold=10**-5,  img_per_class = 200)

Classes used in expt: [0, 1, 2, 3, 4, 5, 6, 7]
Trained & Saved 6 models
Trained & Saved 6 models
Trained & Saved 6 models
Trained & Saved 6 models
Trained & Saved 6 models
Trained & Saved 6 models
Trained & Saved 6 models
Trained & Saved 6 models


 31%|███▏      | 3710/11847 [13:58<30:48,  4.40it/s] 

In [67]:
val_scores
oracle_val_scores

[43.75,
 57.499998807907104,
 68.75,
 80.0000011920929,
 82.4999988079071,
 87.5,
 85.00000238418579,
 88.7499988079071,
 92.5000011920929,
 91.25000238418579]

In [68]:
df = pd.DataFrame([val_scores,oracle_val_scores,pl_accuracies], columns=[f'Iteration {i}' for i in range(len(val_scores))], index=['Our Results','Oracle/Skyline',"Pseudo Labelled"])

In [69]:
df

,Iteration 0,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9
Our Results,44.999999,58.749998,62.50,63.749999,61.250001,64.999998,67.500001,62.500000,66.250002,66.250002
Oracle/Skyline,43.750000,57.499999,68.75,80.000001,82.499999,87.500000,85.000002,88.749999,92.500001,91.250002
